In [1]:
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# Load datasets
data_path = "data"
file_names = {
    "bitcoin": "bitcoin.csv",
    "gold": "gold.csv",
    "google_trends": "google_trends.csv",
    "sp500": "sp500.csv",
    "treasury_3m": "treasury_3m.csv",
    "treasury_10y": "treasury_10y.csv",
    "copper": "copper.csv",
    "oil": "oil.csv",
    "unemployment": "unemployment.csv",
    "moex": "MOEX.csv",
    "sse": "SSE.csv",
    "stoxx": "STOXX_600.csv"
}

data = {}
for key, file in file_names.items():
    file_path = os.path.join(data_path, file)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)

        # Standardize all column names
        df.columns = df.columns.str.strip().str.lower()

        # Rename date column to timestamp
        if "date" in df.columns:
            df.rename(columns={"date": "timestamp"}, inplace=True)

        if "timestamp" not in df.columns:
            continue  # Skip file if no usable timestamp column

        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df.replace({'.': np.nan}, inplace=True)

        for col in df.columns:
            if col != "timestamp":
                df[col] = pd.to_numeric(df[col], errors='coerce')

        data[key] = df


# Custom fix for unemployment column name
if "unemployment" in data:
    unemployment_df = data["unemployment"]
    unemployment_df.columns = unemployment_df.columns.str.strip().str.lower()
    if "unemployment" in unemployment_df.columns:
        unemployment_df.rename(columns={"unemployment": "unemployment_rate"}, inplace=True)
    data["unemployment"] = unemployment_df

rename_map = {
    "bitcoin": {
        "open": "bitcoin_open", "high": "bitcoin_high", "low": "bitcoin_low",
        "close": "bitcoin_close", "volume": "bitcoin_volume"
    },
    "gold": {
        "open": "gold_open", "high": "gold_high", "low": "gold_low",
        "close": "gold_close", "volume": "gold_volume"
    },
    "oil": {
        "open": "oil_open", "high": "oil_high", "low": "oil_low",
        "close": "oil_close", "volume": "oil_volume"
    },
    "sp500": {
        "close": "sp500_close"  # already done, keep this
    },
    "copper": {"price": "copper_price"},
    "google_trends": {
        "spx": "google_spx", "etf": "google_etf",
        "index fund": "google_index_fund", "sp500": "google_sp500"
    },
    "unemployment": {"unemployment": "unemployment_rate"},
    "treasury_3m": {"close": "treasury_3m"},
    "treasury_10y": {"close": "treasury_10y"},
    "moex": {"close": "moex_close"},
    "sse": {"close": "sse_close"},
    "stoxx": {"close": "stoxx_close"}
}



for key, renames in rename_map.items():
    if key in data:
        data[key] = data[key].rename(columns=renames)

# Merge and clean data
sp500 = data["sp500"]
all_data = sp500[["timestamp", "sp500_close"]]
for key, df in data.items():
    if key != "sp500":
        all_data = all_data.merge(df, on="timestamp", how="left")

all_data.sort_values("timestamp", inplace=True)
all_data.fillna(method="ffill", inplace=True)
all_data.dropna(inplace=True)
all_data = all_data.loc[:, all_data.nunique() > 1]

# Feature Engineering
all_data['sp500_lag_1'] = all_data['sp500_close'].shift(1)
all_data['oil_lag_1'] = all_data['oil_close'].shift(1)
all_data['unemp_lag_1'] = all_data['unemployment_rate'].shift(1)

all_data['sp500_rolling_mean_7'] = all_data['sp500_close'].rolling(7).mean()
all_data['sp500_rolling_std_30'] = all_data['sp500_close'].rolling(30).std()

all_data['oil_x_unemployment'] = all_data['oil_close'] * all_data['unemployment_rate']

all_data['month'] = all_data['timestamp'].dt.month
all_data['sin_month'] = np.sin(2 * np.pi * all_data['month'] / 12)
all_data['cos_month'] = np.cos(2 * np.pi * all_data['month'] / 12)

all_data.dropna(inplace=True)

# Define target and features
target = "sp500_close"
exclude_cols = ["timestamp", target]
exog_vars = [col for col in all_data.columns if col not in exclude_cols]

# Monthly SARIMAX backtest
results = []
start_date = pd.Timestamp("2023-01-01")
end_date = pd.Timestamp("2025-03-01")
current_date = start_date

while current_date <= end_date:
    next_month = current_date + pd.DateOffset(months=1)

    train = all_data[all_data["timestamp"] < current_date]
    test = all_data[(all_data["timestamp"] >= current_date) & (all_data["timestamp"] < next_month)]

    if len(train) < 100 or len(test) < 10:
        current_date += pd.DateOffset(months=1)
        continue

    y_train = train[target]
    y_test = test[target]
    X_train = train[exog_vars]
    X_test = test[exog_vars]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    pca = PCA(n_components=0.95)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    best_aic = np.inf
    best_model = None
    best_order = None
    best_seasonal_order = None

    for p in [0, 1]:
        for q in [0, 1]:
            for P in [0, 1]:
                for Q in [0, 1]:
                    try:
                        model = SARIMAX(
                            y_train,
                            exog=X_train_pca,
                            order=(p, 1, q),
                            seasonal_order=(P, 0, Q, 12),
                            enforce_stationarity=False,
                            enforce_invertibility=False
                        )
                        model_fit = model.fit(disp=False)
                        if model_fit.aic < best_aic:
                            best_aic = model_fit.aic
                            best_model = model_fit
                            best_order = (p, 1, q)
                            best_seasonal_order = (P, 0, Q, 12)
                    except:
                        continue

    if best_model:
        forecast = best_model.forecast(steps=len(X_test_pca), exog=X_test_pca)
        r2 = r2_score(y_test, forecast)
        mae = mean_absolute_error(y_test, forecast)
        rmse = np.sqrt(mean_squared_error(y_test, forecast))

        results.append({
            "Backtest": len(results) + 1,
            "Train Size": len(train),
            "Test Start": current_date.strftime("%Y-%m-%d"),
            "R2": r2,
            "MAE": mae,
            "RMSE": rmse,
            "Order": best_order,
            "Seasonal Order": best_seasonal_order,
            "AIC": best_aic
        })

    current_date += pd.DateOffset(months=1)

# Save and print results
results_df = pd.DataFrame(results)
os.makedirs("results", exist_ok=True)
results_df.to_csv("results/sarimax_backtest_with_features.csv", index=False)
print(results_df)




    Backtest  Train Size  Test Start        R2        MAE       RMSE  \
0          1         221  2023-01-01 -1.433869  11.504957  12.209856   
1          2         241  2023-02-01 -1.970673   9.194563  11.410468   
2          3         260  2023-03-01  0.597236   3.102336   3.733481   
3          4         283  2023-04-01 -1.717406   4.003870   4.915230   
4          5         302  2023-05-01 -0.851299   4.590791   5.319414   
5          6         324  2023-06-01 -3.291608  10.810328  11.269171   
6          7         345  2023-07-01  0.044578   5.366106   5.966788   
7          8         365  2023-08-01  0.225459   4.227102   4.772658   
8          9         388  2023-09-01  0.826256   2.821532   3.613066   
9         10         408  2023-10-01 -0.558184   7.175192   9.513455   
10        11         430  2023-11-01  0.654247   5.339988   5.814932   
11        12         451  2023-12-01  0.471318   4.545192   5.428208   
12        13         471  2024-01-01 -1.724036  10.772284  11.95

In [ ]:
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import logging
import warnings
warnings.filterwarnings("ignore")

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Load datasets
data_path = "data"
file_names = {
    "bitcoin": "bitcoin.csv",
    "gold": "gold.csv",
    "google_trends": "google_trends.csv",
    "sp500": "sp500.csv",
    "treasury_3m": "treasury_3m.csv",
    "treasury_10y": "treasury_10y.csv",
    "copper": "copper.csv",
    "oil": "oil.csv",
    "unemployment": "unemployment.csv",
    "moex": "MOEX.csv",
    "sse": "SSE.csv",
    "stoxx": "STOXX_600.csv",
    "vix": "vix.csv",
    "spgsci": "spgsci.csv"
}

data = {}
for key, file in file_names.items():
    file_path = os.path.join(data_path, file)
    if os.path.exists(file_path):
        logging.info(f"Loading {file}")
        df = pd.read_csv(file_path)
        if "timestamp" not in df.columns:
            logging.warning(f"{file} skipped: no 'timestamp' column")
            continue
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df.replace({'.': np.nan}, inplace=True)
        for col in df.columns:
            if col != "timestamp":
                df[col] = pd.to_numeric(df[col], errors='coerce')
        data[key] = df
    else:
        logging.warning(f"{file} not found")

# Rename columns
rename_map = {
    "bitcoin": {"Close": "bitcoin_close", "Open": "bitcoin_open", "High": "bitcoin_high", "Low": "bitcoin_low", "Volume": "bitcoin_volume"},
    "gold": {"Close": "gold_close", "Open": "gold_open", "High": "gold_high", "Low": "gold_low", "Volume": "gold_volume"},
    "oil": {"Close": "oil_close", "Open": "oil_open", "High": "oil_high", "Low": "oil_low", "Volume": "oil_volume"},
    "copper": {"price": "copper_price"},
    "google_trends": {"SPX": "google_spx", "ETF": "google_etf", "index fund": "google_index_fund", "sp500": "google_sp500"},
    "unemployment": {"Unemployment": "unemployment_rate"},
    "treasury_3m": {"Close": "treasury_3m"},
    "treasury_10y": {"Close": "treasury_10y"},
    "sp500": {"Close": "sp500_close"},
    "moex": {"Close": "moex_close"},
    "sse": {"Close": "sse_close"},
    "stoxx": {"Close": "stoxx_close"},
    "vix": {"Close": "vix_close"},
    "spgsci": {"Close": "spgsci_close"}
}

for key, renames in rename_map.items():
    if key in data:
        data[key] = data[key].rename(columns=renames)

# Merge datasets
logging.info("Merging all dataframes")
sp500 = data["sp500"]
all_data = sp500[["timestamp", "sp500_close"]]
for key, df in data.items():
    if key != "sp500":
        all_data = all_data.merge(df, on="timestamp", how="left")

all_data.sort_values("timestamp", inplace=True)
all_data.fillna(method="ffill", inplace=True)
all_data.dropna(inplace=True)

# Advanced Feature Engineering
def add_advanced_features(df, base_cols, lags=[1, 3, 7], windows=[3, 7]):
    for col in base_cols:
        for lag in lags:
            df[f"{col}_lag_{lag}"] = df[col].shift(lag)
        
        for window in windows:
            roll_mean = df[col].rolling(window).mean()
            roll_std = df[col].rolling(window).std()
            roll_min = df[col].rolling(window).min()
            roll_max = df[col].rolling(window).max()
            roll_skew = df[col].rolling(window).skew()
            roll_kurt = df[col].rolling(window).kurt()

            df[f"{col}_roll_mean_{window}"] = roll_mean
            df[f"{col}_roll_std_{window}"] = roll_std
            df[f"{col}_roll_min_{window}"] = roll_min
            df[f"{col}_roll_max_{window}"] = roll_max
            df[f"{col}_roll_skew_{window}"] = roll_skew
            df[f"{col}_roll_kurt_{window}"] = roll_kurt

            # Z-score
            df[f"{col}_zscore_{window}"] = (df[col] - roll_mean) / roll_std

        # Percentage change
        df[f"{col}_pct_change_1"] = df[col].pct_change(1)
    
    return df

key_features = ["sp500_close", "vix_close", "bitcoin_close", "oil_close"]
all_data = add_advanced_features(all_data, key_features)
all_data.dropna(inplace=True)
all_data = all_data.loc[:, all_data.nunique() > 1]
logging.info("Final dataset shape with advanced features: %s", all_data.shape)

# Define target and features
target = "sp500_close"
exclude_cols = ["timestamp", target]
exog_vars = [col for col in all_data.columns if col not in exclude_cols]

# SARIMAX Backtesting
results = []
start_date = pd.Timestamp("2023-01-01")
end_date = pd.Timestamp("2025-03-01")
current_date = start_date

while current_date <= end_date:
    logging.info(f"Backtest for month starting {current_date.strftime('%Y-%m-%d')}")
    next_month = current_date + pd.DateOffset(months=1)

    train = all_data[all_data["timestamp"] < current_date]
    test = all_data[(all_data["timestamp"] >= current_date) & (all_data["timestamp"] < next_month)]

    if len(train) < 100 or len(test) < 10:
        logging.warning("Skipping month due to insufficient data")
        current_date += pd.DateOffset(months=1)
        continue

    y_train = train[target]
    y_test = test[target]
    X_train = train[exog_vars]
    X_test = test[exog_vars]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    pca = PCA(n_components=0.95)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    best_aic = np.inf
    best_model = None
    best_order = None
    best_seasonal_order = None

    for p in [0, 1]:
        for q in [0, 1]:
            for P in [0, 1]:
                for Q in [0, 1]:
                    try:
                        model = SARIMAX(
                            y_train,
                            exog=X_train_pca,
                            order=(p, 1, q),
                            seasonal_order=(P, 0, Q, 12),
                            enforce_stationarity=False,
                            enforce_invertibility=False
                        )
                        model_fit = model.fit(disp=False)
                        if model_fit.aic < best_aic:
                            best_aic = model_fit.aic
                            best_model = model_fit
                            best_order = (p, 1, q)
                            best_seasonal_order = (P, 0, Q, 12)
                    except:
                        continue

    if best_model:
        forecast = best_model.forecast(steps=len(X_test_pca), exog=X_test_pca)
        r2 = r2_score(y_test, forecast)
        mae = mean_absolute_error(y_test, forecast)
        rmse = np.sqrt(mean_squared_error(y_test, forecast))

        results.append({
            "Backtest": len(results) + 1,
            "Train Size": len(train),
            "Test Start": current_date.strftime("%Y-%m-%d"),
            "R2": r2,
            "MAE": mae,
            "RMSE": rmse,
            "Order": best_order,
            "Seasonal Order": best_seasonal_order,
            "AIC": best_aic
        })

    current_date += pd.DateOffset(months=1)

# Save Results
results_df = pd.DataFrame(results)
os.makedirs("results", exist_ok=True)
results_df.to_csv("results/sarimax_backtest_advanced_features.csv", index=False)
logging.info("Saved results to results/sarimax_backtest_advanced_features.csv")
print(results_df)